In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import datetime
import time
from fake_useragent import UserAgent 
ua = UserAgent()
headers = {'user-agent': ua.random}

#data = requests.get(url, headers=headers)

#### Get Monthly Sales for 2019 & 2020

In [2]:
def get_monthly_sales_df(url, year_str, tbl_num):
    '''
    Return a time series df of monthly US car sales for every make and model 
    '''
    response = requests.get(url, headers=headers)
    page= response.text
    soup = BeautifulSoup(page, "lxml")
    
    #find car sales data and turn it into a list
    tbl_str = soup.find_all('tbody')[tbl_num].text
    tbl_list = tbl_str.split('\n')
    tbl_list = [i for i in tbl_list if i] 
    
    #create empty time series df for specified year
    date_time_str = year_str + '-01'
    start_date = datetime.datetime.strptime(date_time_str, '%Y-%m')
    index = pd.date_range(start_date, periods=12, freq='m')
    df = pd.DataFrame(index=index)
    
    #fill empty df with monthly sales for all makes and models
    col_name = ''
    idx = 0
    for x in range(0,len(tbl_list)//13):
        col_name = tbl_list[x+idx]
        list_vals = []
        for val in range(x+1,x+13):
            list_vals.append(tbl_list[idx+val])
        df[col_name]=list_vals
        idx = idx+12
    return df

new df called monthly_sales_df

In [3]:
url = 'https://www.goodcarbadcar.net/2019-us-vehicle-sales-figures-by-model/' 
df_sales = get_monthly_sales_df(url, '2019', 2)
url = 'https://www.goodcarbadcar.net/2020-us-vehicle-sales-figures-by-model/'
df = get_monthly_sales_df(url, '2020', 1)
monthly_sales_df = pd.concat((df_sales, df))
#monthly_sales_df.columns

#### Get All Model Yearly Sales for 2005-2020

In [4]:
def get_model_sales_df(url, year):
    '''
    Return a df of yearly US car sales for every make and model 
    columns = Model, Year, and Total_Sales
    '''
    #read url page into list of pandas dfs 
    response = requests.get(url, headers=headers)
    page = response.text
    df_list = pd.read_html(page)
    
    #find correct df based on number of data frames on url page and 
    if len(df_list) == 1 or year == '2019':
        df = pd.DataFrame(df_list[0])
    elif len(df_list) == 2:
        df = pd.DataFrame(df_list[1])
    else:
        df = pd.DataFrame(df_list[len(df_list)-1])
    
    #special case for 2020 data because sales data is in monthly sales format for each model
    if year == '2020':  
        soup = BeautifulSoup(page, "lxml")
    
        #find car sales data and turn it into a list
        tbl_str = soup.find_all('tbody')[1].text
        tbl_list = tbl_str.split('\n')
        tbl_list = [i for i in tbl_list if i] 
        
        columns = ('Model', 'Year', 'Total_Sales')
        df = pd.DataFrame(columns=columns)

        #fill empty df with monthly sales for all makes and models
        model_name = ''
        idx = 0
        index = 0
        for x in range(0,len(tbl_list)//13):
            model_name = tbl_list[x+idx]
            year_sum = 0
            for val in range(x+1,x+13):
                month_total = tbl_list[idx+val].replace(',','')
                month_total = int(month_total)
                year_sum = year_sum + month_total
            #df1 = pd.DataFrame([model_name, year_sum], columns = ['Model', year])
            df = df.append(pd.DataFrame({'Model': model_name, 'Year': year, 'Total_Sales': year_sum}, index=[index]), 
                           ignore_index=True)
            index = index+1
            #df.append(df1)
            idx = idx+12
        return df
    
    if year == '2012':
        df = df.iloc[:, [2,3]]
    elif year == '2005':
        df = df.iloc[:, [1,3]]
    elif year in ['2017','2018', '2019', '2020']:
        df = df.iloc[:, [0,4]]
    else:
        df = df.iloc[:, [1,2]]
    df.columns = ['Model', 'Total_Sales']
    df['Year'] = year
    
    #* indicate further breakdown of sum totals, overall totals be removed further down when duplicates are removed
    #other symbols refer to subnotes in the tables and are not apart of model names
    symbols = ['*', '²', '¹', '^', '†', '‡']
    
    for s in symbols:
        df['Model'] = df['Model'].str.replace(s,'')

    df.dropna(subset=['Model'], inplace=True)
    
    df['Model'] = df['Model'].str.rstrip()
    df['Model'] = df['Model'].str.lstrip()
    
    #remove first in set of duplicates b/c first is a sum of a car and the hybrid model
    
    df.drop_duplicates(subset='Model', keep='last', inplace=True)
    
    df['Total_Sales'] = df['Total_Sales'].apply(pd.to_numeric, errors='coerce')
    
    return df

In [5]:
url = 'https://www.goodcarbadcar.net/2006-usa-auto-sales-rankings-by-mode/'
yearly_sales_df= get_model_sales_df(url = url, year = '2005')

<ipython-input-4-a3a3f1582c68>:66: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Model'] = df['Model'].str.replace(s,'')


In [6]:
url_dict = {'2006': 'https://www.goodcarbadcar.net/2006-usa-auto-sales-rankings-by-mode/',
            '2007': 'https://www.goodcarbadcar.net/usa-2007-vehicle-sales-rankings-by-mode/',
            '2008': 'https://www.goodcarbadcar.net/2008-america-auto-sales-rankings-by-mode/',
            '2009': 'https://www.goodcarbadcar.net/usa-auto-sales-rankings-by-model-2009/',
            '2010': 'https://www.goodcarbadcar.net/2010-america-auto-sales-rankings-by-mode/',
            '2011': 'https://www.goodcarbadcar.net/top-268-best-selling-vehicles-2011-year/',
            '2012': 'https://www.goodcarbadcar.net/2012-usa-auto-sales-rankings-by-model7/',
            '2013': 'https://www.goodcarbadcar.net/usa-vehicle-sales-rankings-by-model-december-2013-year-end/',
            '2014': 'https://www.goodcarbadcar.net/usa-all-cars-sales-figures-2014-december-year-end/',
            '2015': 'https://www.goodcarbadcar.net/usa-car-sales-by-model-2015-year-end-december/',
            '2016': 'https://www.goodcarbadcar.net/usa-2016-vehicle-sales-by-model-manufacturer-brand/',
            '2017': 'https://www.goodcarbadcar.net/december-2017-year-end-u-s-passenger-car-sales-rankings-top-171-best-selling-cars-america-every-car-ranked/',
            '2018': 'https://www.goodcarbadcar.net/december-2018-the-best-selling-vehicles-in-america-every-vehicle-ranked/',
            '2019': 'https://www.goodcarbadcar.net/2019-us-vehicle-sales-figures-by-model/',
            '2020': 'https://www.goodcarbadcar.net/2020-us-vehicle-sales-figures-by-model/'
           }

years = ['2006']
for key, value in url_dict.items():
    #print(key)
    years.append(key)
    df= get_model_sales_df(url = value, year = key)
    #print(df)
    yearly_sales_df = pd.concat([yearly_sales_df, df], axis=0)
    #yearly_sales_df = pd.merge(yearly_sales_df, df, on='Model', how='outer')
    #print(yearly_sales_df[yearly_sales_df.Model.str.contains("Toyota")])

print(yearly_sales_df)

<ipython-input-4-a3a3f1582c68>:66: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Model'] = df['Model'].str.replace(s,'')
<ipython-input-4-a3a3f1582c68>:66: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Model'] = df['Model'].str.replace(s,'')
<ipython-input-4-a3a3f1582c68>:66: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Model'] = df['Model'].str.replace(s,'')
<ipython-input-4-a3a3f1582c68>:66: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single chara

                     Model Total_Sales  Year
0            Ford F-Series    901463.0  2005
1      Chevrolet Silverado    705980.0  2005
2             Toyota Camry    431703.0  2005
3    Toyota Corolla/Matrix    341290.0  2005
4                Dodge Ram    400543.0  2005
..                     ...         ...   ...
298        Volvo 60-Series       15729  2020
299        Volvo 90-Series        3195  2020
300             Volvo XC40       23778  2020
301             Volvo XC60       32078  2020
302             Volvo XC90       34251  2020

[4518 rows x 3 columns]


In [7]:
# columns = ('Model','Total_Sales')
# total_sales_df = pd.DataFrame(columns=columns)
# total_sales_df = total_sales_df.fillna(0)
# total_sales_df['Model'] = yearly_sales_df['Model']

# total_sales_df['Total_Sales'] = yearly_sales_df[years].sum(axis=1)
# #print(total_sales_df)

In [8]:
# total_sales_df.sort_values('Total_Sales', ascending=False, inplace=True)
yearly_sales_df = yearly_sales_df[yearly_sales_df.Total_Sales != 0]

remove_strings = ["Market",'Total','Family','Brand','Passenger Cars, SUVs, Crossovers','Minivans','Pickup Trucks',
                 'Commercial Vans', 'COMPANY', 'MOTOR', 'GROUP', 'AMERICAN', 'AUTOMOBILES', 'JAGUAR', 'DAIMLER']

for string in remove_strings:
    yearly_sales_df = yearly_sales_df[~yearly_sales_df.Model.str.contains(string)]

#total_sales_df = total_sales_df[total_sales_df.Model != '*Market*']
yearly_sales_df = yearly_sales_df.reset_index(drop=True)
yearly_sales_df

,Model,Total_Sales,Year
0,Ford F-Series,901463.0,2005
1,Chevrolet Silverado,705980.0,2005
2,Toyota Camry,431703.0,2005
3,Toyota Corolla/Matrix,341290.0,2005
4,Dodge Ram,400543.0,2005
...,...,...,...
4388,Volvo 60-Series,15729,2020
4389,Volvo 90-Series,3195,2020
4390,Volvo XC40,23778,2020
4391,Volvo XC60,32078,2020


In [13]:
def get_brand_links(org_url):
    '''
    Return a df of yearly US car sales for every make and model 
    columns = Model, Year, and Total_Sales
    '''
    #org_url = 'https://www.carspecs.us/'
    response = requests.get(org_url, headers=headers)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    #tbl = soup.find_all(class_='modelnamesandmakes_item')
    #link_list = soup.find_all(id_ = 'homepage-browsemakes')
    brand_link_dict = dict()
    link_list = soup.find_all('li')
    for x in link_list[7:]:
        brand = x.text
        link = 'https://www.carspecs.us' + x.find('a')['href']
        brand_link_dict[brand] = link
        time.sleep(0.5)
    return brand_link_dict

In [21]:
def get_model_links(brand, url):
    '''
    Return a df of yearly US car sales for every make and model 
    columns = Model, Year, and Total_Sales
    '''
    #load url page with all car models for specified brand url
    response = requests.get(url, headers=headers) #random user agent
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    
    #find first div with all model links
    div_list = soup.find_all('div', class_='pure-u-1 pure-u-md-1-2')
    
    #new df to store all makes and models for all years past 2004
#     columns = ('Model', 'Year', 'Model_url')
#     df = pd.DataFrame(columns=columns)
    model_link_dict = dict()
    
    #loop through all model links for specified brand 
    for x in div_list[1].find_all('li'):
        model = brand + ' ' + x.text 
        link = org_url + x.find('a')['href'] #model url
        
        #load model url to get list of model year urls
        response = requests.get(link)
        page = response.text
        soup = BeautifulSoup(page, "lxml")
        year_links = soup.find_all('li') #list of all year urls for specified model
        
        #loop through all year links starting at index 7
        idx = 7
#         index = 0
        for y in year_links[7:]:
            year = year_links[idx].text
            if year in years: #only add year links that are 2005 to 2020
                print(year)
                model_link = 'https://www.carspecs.us' + year_links[idx].find('a')['href']
#                 df = df.append(pd.DataFrame({'Model': model, 'Year': year, 'Model_url': model_link}, index=[index]), 
#                                ignore_index=True)
#                 index = index+1
                model_link_dict[model] = (year, model_link)
                idx = idx+1
        time.sleep(0.5) #pause 
            

    return model_link_dict

In [22]:
def get_model_specs(url):
#url = 'https://www.carspecs.us//cars/2020/acura/mdx'
    response = requests.get(url, headers=headers)
    page = response.text
    soup = BeautifulSoup(page, "lxml")

    doors =passengers =speed =horsepower =drive =engine =tank =volume =length =width =height = float('NaN')
    mpg = ''
    try:
        price = soup.find(text=re.compile('starting from'))
        price = price.findNext().text
    
    except:
        price = float('NaN')
    
    div_list = soup.find('div', class_='car-details').find_all('div')
    #print(div_list)

    for div in div_list[1:]:
        spec_list = div.text.split('\n')
        spec_list = [i for i in spec_list if i]
        #print(spec_list)
        if spec_list:
            if 'RPM' not in spec_list[0]:
                if 'Passenger Doors' in spec_list[0]:
                    doors = int(spec_list[-1])
                if 'Passenger Capacity' in spec_list[0]:
                    passengers = int(spec_list[-1])
                if 'mph' in spec_list[0]:
                    speed = spec_list[-1]
                if 'Horsepower' in spec_list[0]:
                    horsepower = spec_list[-1]
                if 'Drive type' in spec_list[0]:
                    drive = spec_list[-1]
                if 'combined' in spec_list[0]:
                    mpg = spec_list[-1]
                if 'Combined' in spec_list[0]:
                    mpg = spec_list[-1]
                if 'Engine type' in spec_list[0]:
                    engine = spec_list[-1]
                    engine = engine.replace('\t', '')
                if 'tank capacity' in spec_list[0]:
                    tank = spec_list[-1]
                if 'EPA interior' in spec_list[0]:
                    volume = spec_list[-1]
                if 'Length' in spec_list[0]:
                    length = spec_list[-1]
                if 'Width' in spec_list[0]:
                    width = spec_list[-1]
                if 'Height' in spec_list[0]:
                    height = spec_list[-1]
        
        if mpg == '':
            mpg = soup.find(text=re.compile('highway mpg'))
            try:
                mpg_list = mpg.split('/ ')
                city = mpg_list[0].lstrip('\r\n ')
                highway = mpg_list[1]
                mpg = (int(city[0:2])+int(highway[0:2]))/2
            except:
                mpg = 0
    
    return [price, doors, passengers, speed, horsepower, drive, mpg, engine, tank, volume, length, width, height]

In [24]:
org_url = 'https://www.carspecs.us/'
brand_links = get_brand_links(org_url)
#print(brand_links)


In [25]:
all_model_links = dict()
for key, value in brand_links.items():
    all_model_links.update(get_model_links(key, brand_links[key]))
    #print(all_model_links)
#rint(all_model_links)

In [17]:
columns = ('Model', 'url', 'price', 'doors', 'passengers', 'speed_sec', 'horsepower_hp', 'drive', 'mpg', 'engine', 
           'tank_gal', 'volume_cuft', 'length_in', 'width_in', 'height_in')
model_spec_df = pd.DataFrame(columns=columns) 
idx = 0
for key, value in all_model_links.items():
    print(value)
    specs = get_model_specs(value[1])
    print(specs)
    print(value[0])
    print(value[1])
    model_spec_df = model_spec_df.append(pd.DataFrame({'Model': key,'Year':value[0], 'url': value[1], 'price': specs[0],'doors': specs[1],
                                                      'passengers': specs[2],'speed_sec': specs[3], 'horsepower_hp': specs[4],
                                                       'drive': specs[5],'mpg': specs[6],'engine': specs[7],
                                                       'tank_gal': specs[8], 'volume_cuft': specs[9], 'length_in': specs[10],
                                                      'width_in': specs[11],'height_in': specs[12]}, index=[idx]), ignore_index=True)
    idx = idx +1
    #print(model_spec_df)

(2013, 'https://www.carspecs.us/cars/2013/acura/ilx')
[nan, nan, nan, '8.98 sec', '150 hp', ' Front wheel drive  ', 29.5, 'Gas', '13.20 gal.', '101.70 cu.ft.', '179.10 in.', '70.60 ', '55.60 in.']
2013
https://www.carspecs.us/cars/2013/acura/ilx
(2014, 'https://www.carspecs.us/cars/2014/acura/ilx-hybrid')
[nan, nan, nan, '11.40 sec', '111 hp', ' Front wheel drive  ', 38.5, 'Hybrid', '13.20 gal.', '99.30 cu.ft.', '179.10 in.', '70.60 ', '55.60 in.']
2014
https://www.carspecs.us/cars/2014/acura/ilx-hybrid
(2005, 'https://www.carspecs.us/cars/2005/acura/mdx')
['$5,535', nan, nan, '7.16 sec', '265 hp', ' All wheel drive  ', 18.0, 'Gas', '20.40 gal.', nan, '188.70 in.', '77.00 ', '68.70 in.']
2005
https://www.carspecs.us/cars/2005/acura/mdx
(2005, 'https://www.carspecs.us/cars/2005/acura/nsx')
[nan, nan, nan, '5.01 sec', '290 hp', ' Rear wheel drive  ', 19.0, 'Gas', '18.50 gal.', '53.90 cu.ft.', '174.20 in.', '71.30 ', '46.10 in.']
2005
https://www.carspecs.us/cars/2005/acura/nsx
(2007, 'ht

['$3,958', nan, nan, '7.34 sec', '200 hp', ' Front wheel drive  ', 25.0, 'Gas', '14.50 gal.', nan, '168.70 in.', '69.50 ', '56.00 in.']
2006
https://www.carspecs.us/cars/2006/audi/a3
(2017, 'https://www.carspecs.us/cars/2017/audi/a3-cabriolet')
['$37,600', 2, 4, nan, '186 hp', 'Front Wheel Drive ', '28 mpg', 'Gas', '13.20 gal.', nan, '174.10 in.', '7.50 in.', '55.50 in.']
2017
https://www.carspecs.us/cars/2017/audi/a3-cabriolet
(2016, 'https://www.carspecs.us/cars/2016/audi/a3-e-tron')
['$30,699', 4, 5, '8.40 sec', '204 hp', 'Front Wheel Drive ', '83 mpg', 'Gas', '10.60 gal.', nan, '169.80 in.', '70.30 ', '56.00 in.']
2016
https://www.carspecs.us/cars/2016/audi/a3-e-tron
(2017, 'https://www.carspecs.us/cars/2017/audi/a3-sedan')
['$31,200', 4, 5, nan, '186 hp', 'Front Wheel Drive ', '29 mpg', 'Gas', '13.20 gal.', nan, '175.50 in.', '70.70 ', '55.70 in.']
2017
https://www.carspecs.us/cars/2017/audi/a3-sedan
(2016, 'https://www.carspecs.us/cars/2016/audi/a3-sportback-e-tron')
[nan, nan, n

['$53,100', 2, 4, '5.03 sec', '333 hp', 'All Wheel Drive ', '20 mpg', 'Gas', '16.10 gal.', nan, '182.70 in.', '73.00 ', '53.90 in.']
2017
https://www.carspecs.us/cars/2017/audi/s5-coupe
(2018, 'https://www.carspecs.us/cars/2018/audi/s5-sportback')
['$54,400', 4, 5, '5.23 sec', '354 hp', 'All Wheel Drive ', '24 mpg', 'Gas', '15.30 gal.', nan, '187.10 in.', '72.60 ', '54.50 in.']
2018
https://www.carspecs.us/cars/2018/audi/s5-sportback
(2007, 'https://www.carspecs.us/cars/2007/audi/s6')
['$12,960', 4, 5, '4.95 sec', '435 hp', 'All Wheel Drive ', 18.0, 'Gas', '21.10 gal.', nan, '193.50 in.', '79.20 ', '57.00 in.']
2007
https://www.carspecs.us/cars/2007/audi/s6
(2013, 'https://www.carspecs.us/cars/2013/audi/s7')
['$38,612', 4, 5, '5.10 sec', '420 hp', 'All Wheel Drive ', '20 mpg', 'Gas', '19.80 gal.', nan, '195.60 in.', '75.20 ', '55.90 in.']
2013
https://www.carspecs.us/cars/2013/audi/s7
(2007, 'https://www.carspecs.us/cars/2007/audi/s8')
['$16,560', 4, 5, '4.66 sec', '450 hp', 'All Wheel

['$96,900', 4, 5, '5.53 sec', '505 hp', 'Rear Wheel Drive ', '13 mpg', 'Gas', '25.40 gal.', nan, '219.50 in.', '75.80 ', '59.90 in.']
2011
https://www.carspecs.us/cars/2011/bentley/mulsanne
(2008, 'https://www.carspecs.us/cars/2008/bmw/1-series')
['$6,951', nan, nan, '6.10 sec', '230 hp', ' Rear wheel drive  ', 23.0, 'Gas', '14.00 gal.', nan, '171.70 in.', '68.80 ', '56.00 in.']
2008
https://www.carspecs.us/cars/2008/bmw/1-series
(2011, 'https://www.carspecs.us/cars/2011/bmw/1-series-m')
['$15,685', 2, 4, '4.64 sec', '335 hp', 'Rear Wheel Drive ', 0, 'Gas', '14.00 gal.', nan, '172.20 in.', '71.00 ', '55.90 in.']
2011
https://www.carspecs.us/cars/2011/bmw/1-series-m
(2008, 'https://www.carspecs.us/cars/2008/bmw/128')
['$8,211', nan, nan, '6.43 sec', '230 hp', 'rear-wheel ', 23.0, nan, '14.00 gal.', nan, nan, nan, nan]
2008
https://www.carspecs.us/cars/2008/bmw/128
(2008, 'https://www.carspecs.us/cars/2008/bmw/135')
['$8,211', nan, nan, '5.46 sec', '300 hp', 'rear-wheel ', 21.5, nan, '14

(2005, 'https://www.carspecs.us/cars/2005/bmw/7-series')
['$10,485', nan, nan, '6.31 sec', '325 hp', ' Rear wheel drive  ', 20.0, 'Gas', '23.30 gal.', '122.00 cu.ft.', '198.00 in.', '74.90 ', '58.70 in.']
2005
https://www.carspecs.us/cars/2005/bmw/7-series
(2014, 'https://www.carspecs.us/cars/2014/bmw/7-series-gran-coupe')
['$66,700', nan, nan, '5.03 sec', '445 hp', ' Rear wheel drive  ', 21.0, 'Gas', '18.50 gal.', '109.00 cu.ft.', '197.20 in.', '74.60 ', '54.80 in.']
2014
https://www.carspecs.us/cars/2014/bmw/7-series-gran-coupe
(2011, 'https://www.carspecs.us/cars/2011/bmw/740')
[nan, nan, nan, '5.73 sec', '315 hp', 'rear-wheel ', 21.0, nan, '21.70 gal.', nan, nan, nan, nan]
2011
https://www.carspecs.us/cars/2011/bmw/740
(2005, 'https://www.carspecs.us/cars/2005/bmw/745')
[nan, nan, nan, '6.31 sec', '325 hp', 'rear-wheel ', 22.0, nan, '23.30 gal.', nan, nan, nan, nan]
2005
https://www.carspecs.us/cars/2005/bmw/745
(2006, 'https://www.carspecs.us/cars/2006/bmw/750')
[nan, nan, nan, '6

...

[nan, nan, nan, '5.98 sec', '200 hp', ' Rear wheel drive  ', 26.0, 'Gas', '13.20 gal.', '83.40 cu.ft.', '166.70 in.', '69.90 ', '50.60 in.']
2013
https://www.carspecs.us/cars/2013/scion/fr-s
(2016, 'https://www.carspecs.us/cars/2016/scion/ia')
['$13,608', 4, 5, '10.14 sec', '106 hp', 'Front Wheel Drive ', '37 mpg', 'Gas', '11.60 gal.', nan, '171.70 in.', '66.70 ', '58.50 in.']
2016
https://www.carspecs.us/cars/2016/scion/ia
(2016, 'https://www.carspecs.us/cars/2016/scion/im')
['$15,552', 4, 5, nan, '137 hp', 'Front Wheel Drive ', '32 mpg', 'Gas', '14.00 gal.', nan, '170.50 in.', '69.30 ', '55.30 in.']
2016
https://www.carspecs.us/cars/2016/scion/im
(2012, 'https://www.carspecs.us/cars/2012/scion/iq')
['$6,106', nan, nan, '10.08 sec', '94 hp', ' Front wheel drive  ', 36.5, 'Gas', '8.50 gal.', '77.30 cu.ft.', '120.10 in.', '66.10 ', '59.10 in.']
2012
https://www.carspecs.us/cars/2012/scion/iq
(2005, 'https://www.carspecs.us/cars/2005/scion/tc')
['$2,400', 2, 5, '7.96 sec', '160 hp', 'Fro

['$2,759', 4, 5, '7.56 sec', '165 hp', 'Rear Wheel Drive ', 20.5, 'Gas', '16.90 gal.', nan, '47.60 in.', '43.50 in.', '67.30 in.']
2005
https://www.carspecs.us/cars/2005/suzuki/grand-vitara
(2010, 'https://www.carspecs.us/cars/2010/suzuki/kizashi')
[nan, nan, nan, '7.75 sec', '185 hp', 'front-wheel ', 24.5, nan, '16.60 gal.', nan, nan, nan, nan]
2010
https://www.carspecs.us/cars/2010/suzuki/kizashi
(2005, 'https://www.carspecs.us/cars/2005/suzuki/reno')
['$2,152', 4, 5, '9.90 sec', '126 hp', 'Front Wheel Drive ', 26.0, 'Gas', '14.50 gal.', nan, '169.10 in.', '67.90 ', '56.90 in.']
2005
https://www.carspecs.us/cars/2005/suzuki/reno
(2007, 'https://www.carspecs.us/cars/2007/suzuki/sx4')
['$2,897', 4, 5, '8.17 sec', '143 hp', 'All Wheel Drive ', 29.5, 'Gas', '11.00 gal.', nan, '162.80 in.', '69.10 ', '63.20 in.']
2007
https://www.carspecs.us/cars/2007/suzuki/sx4
(2005, 'https://www.carspecs.us/cars/2005/suzuki/verona')
['$2,902', 4, 5, '9.81 sec', '155 hp', 'Front Wheel Drive ', 24.0, 'Ga

['$7,580', 4, 5, nan, '73 hp', 'Front Wheel Drive ', '50 mpg', 'Gas', '9.50 gal.', nan, '157.30 in.', '66.70 ', '56.90 in.']
2012
https://www.carspecs.us/cars/2012/toyota/prius-c
(2012, 'https://www.carspecs.us/cars/2012/toyota/prius-plug-in')
['$12,800', nan, nan, '10.41 sec', '134 hp', ' Front wheel drive  ', 0, 'Hybrid', '10.60 gal.', '115.30 cu.ft.', '176.40 in.', '68.70 ', '58.70 in.']
2012
https://www.carspecs.us/cars/2012/toyota/prius-plug-in
(2017, 'https://www.carspecs.us/cars/2017/toyota/prius-prime')
[nan, nan, nan, '11.77 sec', '121 hp', ' Front wheel drive  ', 54.0, 'Hybrid', '11.30 gal.', '111.30 cu.ft.', '182.90 in.', '69.30 ', '57.90 in.']
2017
https://www.carspecs.us/cars/2017/toyota/prius-prime
(2012, 'https://www.carspecs.us/cars/2012/toyota/prius-v')
['$10,620', 4, 5, nan, '98 hp', 'Front Wheel Drive ', '42 mpg', 'Gas', '11.90 gal.', nan, '181.70 in.', '69.90 ', '62.00 in.']
2012
https://www.carspecs.us/cars/2012/toyota/prius-v
(2005, 'https://www.carspecs.us/cars/2

[nan, nan, nan, '7.39 sec', '200 hp', ' Front wheel drive  ', 24.5, 'Gas', '14.50 gal.', '107.00 cu.ft.', '179.30 in.', '70.10 ', '57.40 in.']
2008
https://www.carspecs.us/cars/2008/volkswagen/jetta-gli
(2013, 'https://www.carspecs.us/cars/2013/volkswagen/jetta-hybrid')
[nan, nan, nan, '7.91 sec', '170 hp', ' Front wheel drive  ', 45.0, nan, '11.90 gal.', '105.40 cu.ft.', '182.80 in.', '70.00 ', '57.20 in.']
2013
https://www.carspecs.us/cars/2013/volkswagen/jetta-hybrid
(2005, 'https://www.carspecs.us/cars/2005/volkswagen/jetta-sedan')
['$2,783', 4, 5, '11.06 sec', '115 hp', 'Front Wheel Drive ', 27.0, 'Gas', '14.50 gal.', nan, '172.30 in.', '68.30 ', '56.70 in.']
2005
https://www.carspecs.us/cars/2005/volkswagen/jetta-sedan
(2011, 'https://www.carspecs.us/cars/2011/volkswagen/jetta-sportwagen')
['$6,798', nan, nan, '8.23 sec', '170 hp', ' Front wheel drive  ', 28.0, 'Gas', '14.50 gal.', '124.50 cu.ft.', '179.40 in.', '70.10 ', '59.20 in.']
2011
https://www.carspecs.us/cars/2011/volksw

['$35,700', 4, 5, '6.57 sec', '248 hp', 'All Wheel Drive ', '26 mpg', 'Gas', '14.20 gal.', nan, '34.90 in.', '37.30 in.', '65.30 in.']
2019
https://www.carspecs.us/cars/2019/volvo/xc40
(2010, 'https://www.carspecs.us/cars/2010/volvo/xc60')
['$9,394', nan, nan, '8.16 sec', '235 hp', ' Front wheel drive  ', 22.5, 'Gas', '18.50 gal.', nan, '182.20 in.', '74.40 ', '67.40 in.']
2010
https://www.carspecs.us/cars/2010/volvo/xc60
(2005, 'https://www.carspecs.us/cars/2005/volvo/xc70')
['$5,307', 4, 5, '7.45 sec', '208 hp', 'All Wheel Drive ', 21.0, 'Gas', '18.00 gal.', nan, '33.30 in.', '73.20 ', '61.50 in.']
2005
https://www.carspecs.us/cars/2005/volvo/xc70
(2005, 'https://www.carspecs.us/cars/2005/volvo/xc90')
['$5,293', 4, 5, '9.67 sec', '208 hp', 'Front Wheel Drive ', 20.5, 'Gas', '21.10 gal.', nan, '41.20 in.', '74.70 ', '70.20 in.']
2005
https://www.carspecs.us/cars/2005/volvo/xc90


In [27]:
#model_spec_df['Year'] = model_spec_df['Year'].astype(int)
#model_spec_df['Year'] = model_spec_df['Year'].str.strip()
#yearly_sales_df['Year'] = yearly_sales_df['Year'].astype(int)
#yearly_sales_df['Year'] = yearly_sales_df['Year'].str.strip()
# yearly_sales_df['Model_Merged'] = yearly_sales_df['Model'].str.strip() + yearly_sales_df['Year'].str.strip()
# yearly_sales_df['Model_Merged'] = yearly_sales_df['Model_Merged'].str.lower()
# yearly_sales_df['Model_Merged'] = yearly_sales_df['Model_Merged'].str.replace('-','')
# yearly_sales_df['Model_Merged'] = yearly_sales_df['Model_Merged'].str.replace('/','')
# model_spec_df['Model_Merged'] = model_spec_df['Model'].str.strip() + model_spec_df['Year'].str.strip()
# model_spec_df['Model_Merged'] = model_spec_df['Model_Merged'].str.lower()
# model_spec_df['Model_Merged'] = model_spec_df['Model_Merged'].str.replace('-','')
# model_spec_df['Model_Merged'] = model_spec_df['Model_Merged'].str.replace('/','')
model_spec_df.info()
# model_spec_df['Year'] = model_spec_df['Year'].astype(str)
# yearly_sales_df['Model_Merged'] = yearly_sales_df['Model'].str.strip() + yearly_sales_df['Year'].str.strip()
# yearly_sales_df['Model_Merged'] = yearly_sales_df['Model_Merged'].str.lower()
# yearly_sales_df['Model_Merged'] = yearly_sales_df['Model_Merged'].str.replace('-','')
# yearly_sales_df['Model_Merged'] = yearly_sales_df['Model_Merged'].str.replace('/','')
# model_spec_df['Model_Merged'] = model_spec_df['Model'].str.strip() + model_spec_df['Year'].str.strip()
# model_spec_df['Model_Merged'] = model_spec_df['Model_Merged'].str.lower()
# model_spec_df['Model_Merged'] = model_spec_df['Model_Merged'].str.replace('-','')
# model_spec_df['Model_Merged'] = model_spec_df['Model_Merged'].str.replace('/','')
model_spec_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1240 entries, 0 to 1239
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Model          1240 non-null   object 
 1   url            1240 non-null   object 
 2   price          905 non-null    object 
 3   doors          637 non-null    float64
 4   passengers     637 non-null    float64
 5   speed_sec      1007 non-null   object 
 6   horsepower_hp  1219 non-null   object 
 7   drive          1238 non-null   object 
 8   mpg            1240 non-null   object 
 9   engine         955 non-null    object 
 10  tank_gal       1188 non-null   object 
 11  volume_cuft    220 non-null    object 
 12  length_in      911 non-null    object 
 13  width_in       968 non-null    object 
 14  height_in      970 non-null    object 
 15  Year           1240 non-null   object 
dtypes: float64(2), object(14)
memory usage: 155.1+ KB


,Model,url,price,doors,passengers,speed_sec,horsepower_hp,drive,mpg,engine,tank_gal,volume_cuft,length_in,width_in,height_in,Year
0,Acura ILX,https://www.carspecs.us/cars/2013/acura/ilx,NaN,NaN,NaN,8.98 sec,150 hp,Front wheel drive,29.5,Gas,13.20 gal.,101.70 cu.ft.,179.10 in.,70.60,55.60 in.,2013.0
1,Acura ILX Hybrid,https://www.carspecs.us/cars/2014/acura/ilx-hy...,NaN,NaN,NaN,11.40 sec,111 hp,Front wheel drive,38.5,Hybrid,13.20 gal.,99.30 cu.ft.,179.10 in.,70.60,55.60 in.,2014.0
2,Acura MDX,https://www.carspecs.us/cars/2005/acura/mdx,"$5,535",NaN,NaN,7.16 sec,265 hp,All wheel drive,18.0,Gas,20.40 gal.,NaN,188.70 in.,77.00,68.70 in.,2005.0
3,Acura NSX,https://www.carspecs.us/cars/2005/acura/nsx,NaN,NaN,NaN,5.01 sec,290 hp,Rear wheel drive,19.0,Gas,18.50 gal.,53.90 cu.ft.,174.20 in.,71.30,46.10 in.,2005.0
4,Acura RDX,https://www.carspecs.us/cars/2007/acura/rdx,"$5,939",NaN,NaN,7.05 sec,240 hp,All wheel drive,19.5,Gas,18.00 gal.,NaN,180.70 in.,73.60,65.20 in.,2007.0


In [28]:
#model_spec_df['Year'] = model_spec_df['Year'].astype(str)
#model_spec_sales_df = yearly_sales_df.merge(model_spec_df, how = 'left', on = ['Model_Merged'])
model_spec_sales_df = yearly_sales_df.merge(model_spec_df, how = 'left',on=["Model"])
model_spec_sales_df.info()
model_spec_sales_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4393 entries, 0 to 4392
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Model          4393 non-null   object 
 1   Total_Sales    4310 non-null   object 
 2   Year_x         4393 non-null   object 
 3   url            3841 non-null   object 
 4   price          3455 non-null   object 
 5   doors          2404 non-null   float64
 6   passengers     2404 non-null   float64
 7   speed_sec      3350 non-null   object 
 8   horsepower_hp  3811 non-null   object 
 9   drive          3835 non-null   object 
 10  mpg            3841 non-null   object 
 11  engine         3654 non-null   object 
 12  tank_gal       3730 non-null   object 
 13  volume_cuft    1006 non-null   object 
 14  length_in      3632 non-null   object 
 15  width_in       3740 non-null   object 
 16  height_in      3740 non-null   object 
 17  Year_y         3841 non-null   object 
dtypes: float

,Model,Total_Sales,Year_x,url,price,doors,passengers,speed_sec,horsepower_hp,drive,mpg,engine,tank_gal,volume_cuft,length_in,width_in,height_in,Year_y
0,Ford F-Series,901463.0,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Chevrolet Silverado,705980.0,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Toyota Camry,431703.0,2005,https://www.carspecs.us/cars/2005/toyota/camry,"$3,018",4.0,5.0,8.48 sec,160 hp,Front Wheel Drive,28.5,Gas,18.50 gal.,101.80 cu.ft.,189.20 in.,70.70,58.70 in.,2005.0
3,Toyota Corolla/Matrix,341290.0,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dodge Ram,400543.0,2005,https://www.carspecs.us/cars/2005/dodge/ram,"$4,061",2.0,3.0,NaN,235 hp,Four Wheel Drive,16.0,Gas,26.00 gal.,NaN,NaN,51.00 in.,76.60 in.,2005.0


drop uneeded columns and remove rows with a lot of na values

In [30]:
model_spec_sales_df.drop(labels=['Model', 'Year_y', 'url'], axis=1, inplace=True)
# model_spec_sales_df.dropna(how = ['price', 'doors', 'passengers', 'speed_sec', 'horsepower_hp', 'drive', 'mpg', 
#                                   'engine', 'tank_gal', 'volume_cuft', 'length_in', 'width_in', 'height_in'], inplace=True)

Clean drive feature

In [31]:
model_spec_sales_df['drive'] = model_spec_sales_df['drive'].str.lower()
model_spec_sales_df['drive'] = model_spec_sales_df['drive'].str.lstrip()
model_spec_sales_df['drive'] = model_spec_sales_df['drive'].str.rstrip()
#model_spec_sales_df = model_spec_sales_df.dropna()
model_spec_sales_df['drive'].value_counts()

front wheel drive                           1597
rear wheel drive                             970
all wheel drive                              794
four wheel drive                             389
rear-wheel                                    39
front-wheel                                   20
automatic full-time all wheel                  9
full-time all wheel                            7
real time automatic full-time four-wheel       6
versatrak automatic full-time all wheel        4
Name: drive, dtype: int64

In [37]:
model_spec_sales_df['drive'] = model_spec_sales_df['drive'].str.replace('rear-wheel','rear wheel drive')
model_spec_sales_df['drive'] = model_spec_sales_df['drive'].str.replace('front-wheel','front wheel drive')
#replace = ['full-time all wheel', 'attesa e-ts full-time all wheel', 'automatic full-time all wheel']

model_spec_sales_df['drive'] = model_spec_sales_df['drive'].str.replace('automatic full-time all wheel',
                                                                        'all wheel drive')
model_spec_sales_df['drive'] = model_spec_sales_df['drive'].str.replace('versatrak ','')
model_spec_sales_df['drive'] = model_spec_sales_df['drive'].str.replace('full-time all wheel','all wheel drive')
model_spec_sales_df['drive'] = model_spec_sales_df['drive'].str.replace('real time automatic full-time four-wheel',
                                                                        'all wheel drive')
# for s in replace:
#     model_spec_sales_df['drive'] = model_spec_sales_df['drive'].str.replace(s,'all wheel drive')
model_spec_sales_df['drive'].value_counts()

front wheel drive    1617
rear wheel drive     1009
all wheel drive       820
four wheel drive      389
Name: drive, dtype: int64

In [38]:
model_spec_sales_df.info()
model_spec_sales_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4393 entries, 0 to 4392
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Total_Sales    4310 non-null   object 
 1   Year_x         4393 non-null   object 
 2   price          3455 non-null   object 
 3   doors          2404 non-null   float64
 4   passengers     2404 non-null   float64
 5   speed_sec      3350 non-null   object 
 6   horsepower_hp  3811 non-null   object 
 7   drive          3835 non-null   object 
 8   mpg            3841 non-null   object 
 9   engine         3654 non-null   object 
 10  tank_gal       3730 non-null   object 
 11  volume_cuft    1006 non-null   object 
 12  length_in      3632 non-null   object 
 13  width_in       3740 non-null   object 
 14  height_in      3740 non-null   object 
dtypes: float64(2), object(13)
memory usage: 549.1+ KB


,Total_Sales,Year_x,price,doors,passengers,speed_sec,horsepower_hp,drive,mpg,engine,tank_gal,volume_cuft,length_in,width_in,height_in
0,901463.0,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,705980.0,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,431703.0,2005,"$3,018",4.0,5.0,8.48 sec,160 hp,front wheel drive,28.5,Gas,18.50 gal.,101.80 cu.ft.,189.20 in.,70.70,58.70 in.
3,341290.0,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,400543.0,2005,"$4,061",2.0,3.0,NaN,235 hp,four wheel drive,16.0,Gas,26.00 gal.,NaN,NaN,51.00 in.,76.60 in.


In [39]:
replace_list = ['hp', 'mpg', 'gal.', 'cu.ft.', 'in.', ',', 'sec', '$', '$']
for s in replace_list:
    model_spec_sales_df = model_spec_sales_df.replace(s,'', regex=True) 
model_spec_sales_df['price'] = model_spec_sales_df['price'].str.replace('$','')

<ipython-input-39-eabc4894fe8e>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  model_spec_sales_df['price'] = model_spec_sales_df['price'].str.replace('$','')


In [40]:
num_cols = ['price',  'speed_sec', 'horsepower_hp', 'mpg', 'tank_gal', 'volume_cuft', 
            'width_in', 'length_in','height_in']
for col in num_cols:
    model_spec_sales_df[col] = model_spec_sales_df[col].apply (pd.to_numeric, errors='coerce')
model_spec_sales_df['doors'] = model_spec_sales_df['doors'].astype(str)
model_spec_sales_df['passengers'] = model_spec_sales_df['passengers'].astype(str)

In [41]:
model_spec_sales_df.info()
model_spec_sales_df.sample(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4393 entries, 0 to 4392
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Total_Sales    4310 non-null   float64
 1   Year_x         4393 non-null   object 
 2   price          3455 non-null   float64
 3   doors          4393 non-null   object 
 4   passengers     4393 non-null   object 
 5   speed_sec      3350 non-null   float64
 6   horsepower_hp  3811 non-null   float64
 7   drive          3835 non-null   object 
 8   mpg            3841 non-null   float64
 9   engine         3654 non-null   object 
 10  tank_gal       3730 non-null   float64
 11  volume_cuft    1006 non-null   float64
 12  length_in      3632 non-null   float64
 13  width_in       3740 non-null   float64
 14  height_in      3740 non-null   float64
dtypes: float64(10), object(5)
memory usage: 549.1+ KB


,Total_Sales,Year_x,price,doors,passengers,speed_sec,horsepower_hp,drive,mpg,engine,tank_gal,volume_cuft,length_in,width_in,height_in
3638,173600.0,2018,3300.0,4.0,5.0,7.47,221.0,front wheel drive,25.0,Gas,17.5,NaN,190.2,72.2,28.3
2435,66146.0,2013,3971.0,4.0,5.0,7.69,156.0,front wheel drive,28.0,Gas,13.7,NaN,178.3,69.9,57.5
2617,1598.0,2013,4911.0,4.0,5.0,7.08,250.0,all wheel drive,20.5,Gas,16.9,NaN,44.4,73.9,66.5
3346,59.0,2016,23425.0,nan,nan,5.44,328.0,rear wheel drive,23.0,Gas,20.0,110.0,187.9,69.8,57.2
3114,78565.0,2016,14548.0,4.0,5.0,9.32,138.0,all wheel drive,26.0,Gas,14.0,NaN,168.4,69.9,65.2
1961,9674.0,2011,NaN,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3903,5369.0,2019,NaN,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3271,7019.0,2016,15148.0,nan,nan,10.81,131.0,front wheel drive,25.0,Gas,14.5,NaN,186.3,68.1,73.7
181,20503.0,2005,6285.0,nan,nan,6.73,255.0,all wheel drive,20.5,Gas,21.1,113.8,193.5,71.3,57.5
3198,27812.0,2016,15092.0,nan,nan,NaN,240.0,rear wheel drive,29.0,Gas,16.6,NaN,176.5,70.8,60.8


In [42]:
model_spec_sales_df.to_pickle('../data/model_spec_sales_df.pkl')